Import Data

In [1]:
import wpparser
down_path = "C:\\Users\\kales\\Downloads\\"
#test_data = wpparser.parse(down_path + "tear2023.wordpress.com-2023-12-15-19_01_51-1qoarrtidqhyn2xegrnqyfjiface6xv2\\tear2023.wordpress.com-2023-12-15-19_01_35\\text.xml")
data = wpparser.parse(down_path + "tear2023.wordpress.com-2023-12-15-19_01_51-1qoarrtidqhyn2xegrnqyfjiface6xv2\\tear2023.wordpress.com-2023-12-15-19_01_35\\tear2023.wordpress.2023-12-15.000.xml")

Create Image Data Dictionary 

In [2]:
import re
img_pat = r'files.wordpress.com/(.*?)"'
img_path_re = re.compile(img_pat)
all_img_data = []
for post in data['posts']:
    if post['content'] is None:
        cont = img_path_re.findall(post['guid'] + '\"', re.DOTALL)
        img_data = {
            'id': post['post_id'],
            'width': post['postmeta']['attachment_metadata'][b'width'],
            'height': post['postmeta']['attachment_metadata'][b'height'],
            'path': cont
        }
        all_img_data.append(img_data)

Create Post Content Dictionary

In [17]:
import re

paragraphs = r'<p>(.*?)</p>'
img = r'<img\s+src="https://tear2023.files.wordpress.com/(.*?)\?w='
img_re = re.compile(img)
regex =  r'<p>(.*?)</p>|<img\s+src="https://tear2023.files.wordpress.com/(.*?)\?w='
all_re = re.compile(regex)
post_content = ""
posts_content = []
para_re = re.compile(paragraphs)
for post in data['posts']:
    if (post['content'] is not None):
        fImage_id = ""
        try:
            fImage_id = post['postmeta']['featured_img_id']
        except KeyError:
            fImage_id = ""
        post_content = {
            'title': post['title'],
            'content': all_re.findall(post['content'], re.DOTALL),
            'post_id': post['post_id'],
            'featured_img': fImage_id,
        }
        posts_content.append(post_content)
        #if (post['title'] == "Day 100/101: Finale"):
            #print(post['content'])
            #print(post_content['content'][1][1])
#print(data['posts'][-5])
#print(posts_content[5])

Print Data to Docx

In [15]:
from docx import Document
from docx.shared import Inches
img_path = "C:/Users/kales/Downloads/media-export-220074516-from-0-to-2745/"
doc = Document()
doc.add_heading('Tear 2023', 0)
doc.add_heading('By: Hunter Hatzenbeler, Kale Strange, Sam Shepard, and Sophie Harris', level=1)
for post in posts_content[0:8]:
    #print(post)
    if not ((post['title'] == 'Dinaric Alps Gallery') or (post['title'] == 'Swiss Alps Gallery')):
        doc.add_section()
        doc.add_heading(post['title'], level=1)
        #if len(post['content']) > 1:
        #    if post['content'][1] == "":
        #        builder.writeln(post['content'][0])
        print(post['title'])
        fImagePath = ""
        if not (post['featured_img'] == ""):
            imgID = post['featured_img']
            for img in all_img_data:
                if (img['id'] == imgID):
                    fImagePath = img['path'][0]
                    doc.add_picture((img_path + fImagePath), width=Inches(4))
                    break
        for content in post['content']:
            if content[1] == "":
                doc.add_paragraph(content[0])
            else: 
                #print(content[1])
                if not (content[1] == fImagePath):
                    doc.add_picture((img_path + content[1]), width=Inches(2.5))
#doc.update_fields()
doc.save("out.docx")

About
T-minus 24 Hours
The final supper
Seattle to London ✈️
Shep out
Day 1: Reunions and Bulgaria
